In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample poetry dataset
seed_lines = [
    "Roses are red, violets are blue",
    "I wandered lonely as a cloud",
    "Shall I compare thee to a summer's day",
    "The road not taken, two paths diverged",
    "In Xanadu did Kubla Khan",
    "To be or not to be, that is the question",
    "O Captain! My Captain!",
    "When I consider how my light is spent",
    "Tyger Tyger, burning bright",
    "Do not go gentle into that good night",
    "Ode to a Nightingale",
    "The Raven",
    "Because I could not stop for Death",
    "How do I love thee? Let me count the ways",
    "A thing of beauty is a joy forever",
    "She walks in beauty, like the night",
    "I celebrate myself, and sing myself",
    "Hope is the thing with feathers",
    "I heard a fly buzz when I died",
    "I measure every grief I meet",
    "I never saw a moor",
    "The world is too much with us",
    "The best-laid schemes o' mice an' men",
    "I wandered by the brookside",
    "The time has come, the Walrus said",
    "Not marble, nor the gilded monuments",
    "From fairest creatures we desire increase",
    "My love is like a red, red rose",
    "Water, water, everywhere",
    "A rose by any other name would smell as sweet",
    "The caged bird sings with a fearful trill",
    "Ode to the West Wind",
    "To Autumn",
    "Invictus",
    "Song of Myself",
    "A Dream Within A Dream",
    "Annabel Lee",
    "Jabberwocky",
    "The Second Coming",
    "Stopping by Woods on a Snowy Evening",
    "The Love Song of J. Alfred Prufrock",
    "The sun'll come out tomorrow",
    "Good night, good night! Parting is such sweet sorrow",
    "It was the best of times, it was the worst of times",
    "In the middle of the journey of our life",
    "All that glitters is not gold",
    "Once upon a midnight dreary",
    "Four score and seven years ago",
    "The lady doth protest too much",
    "Out, out, brief candle!",
    "Now is the winter of our discontent",
    "To sleep, perchance to dream",
    "A horse! A horse! My kingdom for a horse!",
    "Cowards die many times before their deaths",
    "The better part of valour, is discretion",
    "The play's the thing",
    "Beware the Ides of March",
    "What's in a name? That which we call a rose",
    "Though this be madness, yet there is method in't",
    "Is this a dagger which I see before me",
    "A little more than kin, and less than kind",
    "All the world's a stage",
    "O Romeo, Romeo! Wherefore art thou Romeo?",
    "Parting is such sweet sorrow",
    "O, had I but followed the arts!",
    "Frailty, thy name is woman",
    "The quality of mercy is not strained",
    "Uneasy lies the head that wears a crown",
    "Something wicked this way comes",
    "To be or not to be, that is the question",
    "The first thing we do, let's kill all the lawyers",
    "We are such stuff as dreams are made on",
    "Some are born great, some achieve greatness",
    "All that lives must die, passing through nature to eternity",
    "The fault, dear Brutus, is not in our stars",
    "The better part of valour, is discretion",
    "It is a tale told by an idiot",
    "That's the long and the short of it",
    "The lady doth protest too much, methinks",
    "All's fair in love and war",
    "Love is blind",
    "To thine own self be true",
    "Too much of a good thing",
    "Fair is foul and foul is fair",
    "Cry havoc and let slip the dogs of war",
    "What's done is done",
    "Friends, Romans, countrymen, lend me your ears",
    "Brevity is the soul of wit",
    "The world is my oyster",
    "What light through yonder window breaks",
    "The green-eyed monster",
    "There are more things in heaven and earth",
    "That's one small step for a man",
    "Elementary, my dear Watson",
    "I think, therefore I am",
    "The unexamined life is not worth living",
    "I am what I am",
    "Cogito, ergo sum",
    "To live is the rarest thing in the world",
    "The only true wisdom is in knowing you know nothing",
    "The mind is its own place, and in itself can make a heaven of hell, a hell of heaven",
    "I know that I am intelligent, because I know that I know nothing",
    "The greatest wealth is to live content with little",
    "Not everything that is faced can be changed",
    "I have nothing to lose but my chains",
    "I have a dream that one day",
    "Our lives begin to end the day we become silent about things that matter",
    "Darkness cannot drive out darkness",
    "The only thing we have to fear is fear itself",
    "I believe that unarmed truth and unconditional love will have the final word",
    "Ask not what your country can do for you",
    "Injustice anywhere is a threat to justice everywhere",
    "Education is the most powerful weapon",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "I can resist anything except temptation",
    "Life imitates art far more than art imitates life",
    "To be great is to be misunderstood",
    "Experience is the name everyone gives to their mistakes",
    "I would die for my country, but I could never let my country die for me",
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages",
    "I don't want to earn my living; I want to live",
    "When the power of love overcomes the love of power, the world will know peace",
    "The future belongs to those who believe in the beauty of their dreams",
    "It always seems impossible until it's done",
    "Life is 10% what happens to us and 90% how we react to it",
    "The only thing standing between you and your dream",
    "The only way to do great work is to love what you do",
    "Don't watch the clock; do what it does. Keep going",
    "Strive not to be a success, but rather to be of value",
    "The greatest glory in living lies not in never falling, but in rising every time we fall",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "The only person you should try to be better than is the person you were yesterday",
    "Success is not final, failure is not fatal: It is the courage to continue that counts",
    "Believe you can and you're halfway there",
    "The biggest risk is not taking any risk",
    "Your time is limited, don't waste it living someone else's life",
    "Do not go where the path may lead, go instead where there is no path and leave a trail",
    "The secret of getting ahead is getting started",
    "It's not whether you get knocked down, it's whether you get up",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work is to love what you do",
    "I can't change the direction of the wind, but I can adjust my sails to always reach my destination",
    "Twenty years from now you will be more disappointed by the things that you didn't do",
    "Keep your face always toward the sunshine - and shadows will fall behind you",
    "The only way to do great work is to love what you do",
    "You miss 100% of the shots you don't take",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "Life is a journey that must be traveled no matter how bad the roads and accommodations",
    "Life is really simple, but we insist on making it complicated",
    "The best revenge is massive success",
    "What lies behind us and what lies before us are tiny matters compared to what lies within us",
    "Do not dwell in the past, do not dream of the future, concentrate the mind on the present moment",
    "The only thing standing between you and your dream is the will to try and the belief that it is actually possible",
    "The only thing we have to fear is fear itself",
    "The most difficult thing is the decision to act, the rest is merely tenacity",
    "Every strike brings me closer to the next home run",
    "The future depends on what you do today",
    "You are never too old to set another goal or to dream a new dream",
    "The only limit to our realization of tomorrow will be our doubts of today",
    "The only person you should try to be better than is the person you were yesterday",
    "It always seems impossible until it is done",
    "I never dream of success, I worked for it",
    "Life is 10% what happens to us and 90% how we react to it",
    "The best time to plant a tree was 20 years ago. The second best time is now",
    "Great things never come from comfort zones",
    "If you are working on something exciting that you really care about, you don't have to be pushed. The vision pulls you",
    "Do not wait for the perfect moment, take the moment and make it perfect",
    "It's not about how hard you hit. It's about how hard you can get hit and keep moving forward",
    "Don't be pushed around by the fears in your mind. Be led by the dreams in your heart",
    "If you want to achieve greatness stop asking for permission",
    "In order to succeed, we must first believe that we can",
    "Success is walking from failure to failure with no loss of enthusiasm",
    "The successful warrior is the average man, with laser-like focus",
    "Opportunities don't happen. You create them",
    "The secret to getting ahead is getting started",
    "Success is liking yourself, liking what you do, and liking how you do it",
    "Success is not the key to happiness. Happiness is the key to success",
    "Don't watch the clock; do what it does. Keep going",
    "The harder I work, the luckier I get",
    "Success is not in what you have, but who you are",
    "The only way to do great work"]

# Tokenize and create sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in poems:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create inputs and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert labels to categorical
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the model
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    LSTM(150),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X, y, epochs=50, verbose=1)

# ...

# Generate poetry
seed_text = "The harder I work,"
next_words = 50
temperature = 1  # Adjust this value (e.g., 0.2 for low randomness, 1.0 for high randomness)

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    predicted_probs = model.predict(token_list, verbose=0)[0]
    predicted_probs = np.log(predicted_probs) / temperature
    exp_preds = np.exp(predicted_probs)
    predicted_probs = exp_preds / np.sum(exp_preds)

    predicted_index = np.random.choice(len(predicted_probs), p=predicted_probs)

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 7, 100)            1800      
                                                                 
 lstm_4 (LSTM)               (None, 150)               150600    
                                                                 
 dense_4 (Dense)             (None, 18)                2718      
                                                                 
Total params: 155,118
Trainable params: 155,118
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 2.8908 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 23ms/step - loss: 2.8813 - accuracy: 0.1176
Epoch 3/50
1/1 [==============================] - 0s 22ms/step - loss: 2.8718 - accuracy: 0.3529
Epoch 4/50
1/1 [===

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample poetry dataset
def load_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

seed_lines = load_data('/content/seed_lines.txt')


# Tokenize and create sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in poems:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create inputs and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert labels to categorical
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the model
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    LSTM(150),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X, y, epochs=50, verbose=1)

# ...

# Generate poetry
seed_text = "The harder I work,"
next_words = 50
temperature = 1  # Adjust this value (e.g., 0.2 for low randomness, 1.0 for high randomness)

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    predicted_probs = model.predict(token_list, verbose=0)[0]
    predicted_probs = np.log(predicted_probs) / temperature
    exp_preds = np.exp(predicted_probs)
    predicted_probs = exp_preds / np.sum(exp_preds)

    predicted_index = np.random.choice(len(predicted_probs), p=predicted_probs)

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 7, 100)            1800      
                                                                 
 lstm_5 (LSTM)               (None, 150)               150600    
                                                                 
 dense_5 (Dense)             (None, 18)                2718      
                                                                 
Total params: 155,118
Trainable params: 155,118
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 2.8911 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 22ms/step - loss: 2.8815 - accuracy: 0.1765
Epoch 3/50
1/1 [==============================] - 0s 26ms/step - loss: 2.8718 - accuracy: 0.2353
Epoch 4/50
1/1 [===